binning the SST for the cloud radar data

In [16]:
# importing necessary libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import glob
from myFunctions import lcl
from myFunctions import f_closest
from warnings import warn
import numpy as np
import pandas as pd
import atmos
import datetime as dt
import matplotlib.pyplot as plt
from scipy import interpolate
import custom_color_palette as ccp
from matplotlib import rcParams
import matplotlib.ticker as ticker


def f_interpolate_SST_and_merge(SST_DS, dataset_obs):
    '''function to interpolate SST values on the time resolution of the observations given as input
    input: 
    SST_DS: xarray dataset containing sst values
    dataset_obs: xarray dataset containing the observations to merge with sst data
    output: 
    data_merged: data returned 
    '''
    
    # interpolating sst data at 1 min resolution to the 10 s res of the wind lidar
    sst_data_interp = SST_DS.interp(time=dataset_obs['time'].values)

    # merging the interpolated dataset and the wind lidar dataset
    data_merged = xr.merge([dataset_obs, sst_data_interp])
    return(data_merged)


def f_calculate_binned_data(data_input, SST_binned_arr):
    
    ''' function to calculate mean values of all variables for each SST bin, for all instruments
    author: Claudia Acquistapace
    date: 20 Sept 2021
    input: - data_input: input xarray dataset containing the variables as a function of time, height, to be averaged
            - SST_binned_arr: numpy array of sst binned values for calculating the mean 
    output: dataset_concat: xarray dataset of concatenated values with mean profiles corresponding to the sst bins. A variable n_el counts the number of profiles averaged together
    '''
    # calculating mean quantities f
    dataset_mean = []

    data_input = data_input.load()

    # selecting all columns in the bin interval
    for ind_bin in range(len(SST_binned_arr)-1):

        # selecting slices of datasets columns with SST values in the selected bin
        DS_sliced = data_merged.where((data_input.SST > SST_binned_arr[ind_bin]) & (data_input.SST < SST_binned_arr[ind_bin+1]), drop=True)

        # add variable of the number of elements of the slice
        n_el = len(DS_sliced.SST.values)
        DS_sliced['n_elements'] = n_el

        # calculate mean profile averaging all selected time stamps together
        dataset_mean.append(DS_sliced.mean(dim='time', skipna=True))


    # concatenating datasets corresponding to SST bins on a new bin dimension
    dataset_concat = xr.concat([dataset_mean[i] for i in np.arange(len(dataset_mean))], dim='SST_binned')
    return(dataset_concat)

def f_calc_tot_cloud_fraction(matrix):
    '''
    function to calculate the total cloud fraction of a matrix (time, height)
    

    Parameters
    ----------
    matrix : ndarray (time, height)
        DESCRIPTION. reflectivity values

    Returns
    -------
    cloud fraction ndarray(time)

    '''
    #defining ndarray to contain cloud fraction
    cloud_fraction = []
    N_tot = matrix.shape[0]
    for ind_height in range(matrix.shape[1]):
        cloud_fraction.append(np.sum(~np.isnan(matrix[:,ind_height]))/N_tot)


    return(np.asarray(cloud_fraction))

In [2]:
# reading data containing flags to filter out rainy columns
flag_file_list = np.sort(glob.glob('/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/*_flags_cloud_properties.nc'))
flag_file_list = flag_file_list[13:15]
print(flag_file_list)
data_1 = xr.open_dataset(flag_file_list[0])
data_2 = xr.open_dataset(flag_file_list[1])
data_1 = data_1.drop('cloud_top_height')
data_2 = data_2.drop('cloud_top_height')
flag_data = xr.merge([data_1, data_2]) #, dim='time')
flag_data['time'] = pd.to_datetime(flag_data.time.values)

['/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/20200202_flags_cloud_properties.nc'
 '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/20200203_flags_cloud_properties.nc']


/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/xarray/core/alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  index = joiner(matching_indexes)


In [3]:
# reading tsg file ( data with 1 min resolution)
tsg_file = "/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/tsg_sst_data/tsg/nc/msm_089_1_tsg.nc"

# opening ship data and reading sst
tsg_data = xr.open_dataset(tsg_file)

# identifying time stamps of sst corresponding to time stamps of radiosondes
t_start = datetime(2020, 2, 2, 0, 0, 0)
t_end = datetime(2020, 2, 3, 23, 59, 59)

# slicing tsg datase t for the selected time interval and extracting sst
sliced_tsg_ds = tsg_data.sel(TIME=slice(t_start, t_end))
tsg_sst = sliced_tsg_ds['TEMP'].values
tsg_time_sst = sliced_tsg_ds['TIME'].values
tsg_flag = sliced_tsg_ds['TEMP_QC'].values

# averaging together the sst of the different sst sensors for tsg
temp0 = sliced_tsg_ds.TEMP[:,0].values
temp1 = sliced_tsg_ds.TEMP[:,1].values
sst_tsg = temp0
sst_tsg[np.isnan(temp0)] = temp1[np.isnan(temp0)]

# producing output dataset of sst_tsg for the selected time window
# creating dataset with coordinates sst and height
dim_sst           = ['time']
coords         = {"time":sliced_tsg_ds.TIME.values}
SST              = xr.DataArray(dims=dim_sst, coords=coords, data=sst_tsg,
                 attrs={'long_name':'sea surface temperature ',
                        'units':'$^{\circ}$C'})
variables         = {'SST':SST}
SST_DS      = xr.Dataset(data_vars = variables,
                       coords = coords)

In [4]:
# building SST binned array
SST_min = np.nanmin(sst_tsg)
SST_max = np.nanmax(sst_tsg)
SST_binned_arr = np.arange(SST_min, SST_max, step=0.025)


# calculate label marks for bins
sst_bin_label = []
for ind in range(len(SST_binned_arr)-1):
    sst_bin_label.append(round((SST_binned_arr[ind]+SST_binned_arr[ind+1])/2,2))
    


In [5]:
# reading radar file list
file_list = np.sort(glob.glob("/Volumes/Extreme SSD/ship_motion_correction_merian/corrected_data/wband_daily_with_DOI/latest/with_DOI/daily_intake/*.nc"))
file_list = file_list[14:16]

# set the processing mode keyword for the data you want to be processed
processing_mode = 'case_study' # 'all_campaign' #

# setting time window to be checked
string_out = '20200202_20200203'
t_start = datetime(2020, 2, 2, 0, 0, 0)
t_end = datetime(2020, 2, 3, 23, 59, 59)


data_1 = xr.open_dataset(file_list[0])
data_2 = xr.open_dataset(file_list[1])
radar_data = xr.merge([data_1, data_2]) 


/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/xarray/core/alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  index = joiner(matching_indexes)


In [13]:
file_list = np.sort(glob.glob("/Volumes/Extreme SSD/ship_motion_correction_merian/corrected_data/wband_daily_with_DOI/latest/with_DOI/daily_intake/*.nc"))
file_list[9:17]
radar_data_dc = xr.open_mfdataset(file_list[9:17])
time_dc = pd.to_datetime(radar_data_dc.time.values)

In [14]:
time_dc

DatetimeIndex(['2020-01-28 00:00:01', '2020-01-28 00:00:04',
               '2020-01-28 00:00:07', '2020-01-28 00:00:10',
               '2020-01-28 00:00:13', '2020-01-28 00:00:17',
               '2020-01-28 00:00:20', '2020-01-28 00:00:23',
               '2020-01-28 00:00:26', '2020-01-28 00:00:29',
               ...
               '2020-02-04 23:59:30', '2020-02-04 23:59:34',
               '2020-02-04 23:59:37', '2020-02-04 23:59:40',
               '2020-02-04 23:59:43', '2020-02-04 23:59:46',
               '2020-02-04 23:59:50', '2020-02-04 23:59:53',
               '2020-02-04 23:59:56', '2020-02-04 23:59:59'],
              dtype='datetime64[ns]', length=213732, freq=None)

In [17]:
# calculating diurnal cycle of the cloud fraction
time_cloud_fraction = np.arange(time_dc[0], time_dc[-1], timedelta(minutes=10))
dim_time_out = len(time_cloud_fraction)
dim_height = np.shape(radar_data_dc.radar_reflectivity.values)[1]
cloud_fraction = np.zeros((dim_time_out, dim_height))

for itime in range(dim_time_out-1):
    data = radar_data_dc.sel(time=slice(time_cloud_fraction[itime], time_cloud_fraction[itime+1]))
    for iheight in range(dim_height):
        cloud_fraction[itime, iheight] = np.count_nonzero(~np.isnan(radar_data_dc.radar_reflectivity.values[:,iheight]))/np.shape(radar_data_dc.radar_reflectivity.values[:,iheight])[0]



KeyboardInterrupt: 

In [ ]:
print()

In [ ]:
# building xarray dataset for cloud fraction to apply then the mean operatore
dims = ['time', 'height']
coords = {'time':time_cloud_fraction, 'height':radar_data_dc.height.values}
cf = xr.DataArray(dims=dims, coords=coords, data=cloud_fraction,
                 attrs={'long_name':'cloud fraction',
                        'units':''})
variables={'cloud_fraction':cf}

cf_dataset =xr.Dataset(data_vars = variables,
                       coords = coords)

In [ ]:

# re-writing time array as hh:mm for then being able to group
cf_dataset['time'] = pd.to_datetime(cf_dataset.time.values).strftime("%H:%M")

# grouping and calculating mean of the profiles
grouped_mean = cf_dataset.groupby('time').mean()
#grouped_std = radar_data_dc.groupby('time').std()


In [ ]:
fig2, axs = plt.subplots(1,1, figsize=(16,7), constrained_layout=True)
axs.spines["top"].set_visible(False)
axs.spines["right"].set_visible(False)
axs.spines["bottom"].set_linewidth(3)
axs.spines["left"].set_linewidth(3)
mesh1 = axs.pcolormesh(pd.to_datetime(grouped_mean['time'].values), grouped_mean['height'].values, \
                             grouped_mean['radar_reflectivity'].values.T, vmin=-50., \
                             vmax=20., cmap='viridis', rasterized=True)
cbar = fig2.colorbar(mesh1, ax=axs, label='Reflectivity mean over 10 min', \
                        location='right', aspect=20, use_gridspec=grid)
axs.set_xlabel('Time UTC [HH:MM]')
axs.set_ylabel('Height [m]')
axs.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
axs.text(0, 1.02, 'Diurnal cycle of '+dict_var['var_string']+' averaged over '+dict_var['avg_time']+' min', \
            fontweight='black', transform=axs.transAxes)
    

In [8]:





# selecting data in the time window of the surface anomaly
radar_data_sliced = radar_data.sel(time=slice(t_start, t_end))


# building a mask to filter out Ze rainy columns and substitute them with nans
# set to nan the values out of the thresholds for the selected variable
mask = np.zeros((len(radar_data_sliced.time.values), len(radar_data_sliced.height.values)))
for ind in range(len(radar_data_sliced.time.values)):
    if (flag_data["flag_rain_ground"].values[ind] == 1) | (flag_data["flag_rain"].values[ind] == 1):
        mask[ind,:] = np.repeat(1, len(radar_data_sliced.height.values))

# filtering rainy values using the mask
radar_data_sliced["nans"] = xr.full_like(radar_data_sliced.radar_reflectivity, fill_value=np.nan)
radar_data_sliced['radar_reflectivity'] = xr.where(mask == 0, radar_data_sliced['radar_reflectivity'], radar_data_sliced["nans"])


/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/xarray/core/alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  index = joiner(matching_indexes)


In [25]:

# calculating cloud fraction
from datetime import timedelta

# building SST binned array
SST_min = np.nanmin(sst_tsg)
SST_max = np.nanmax(sst_tsg)
SST_binned_arr_cf = np.arange(SST_min, SST_max, step=0.05)

# calculate label marks for bins
sst_bin_label_cf = []
for ind in range(len(SST_binned_arr_cf)-1):
    sst_bin_label_cf.append(round((SST_binned_arr_cf[ind]+SST_binned_arr_cf[ind+1])/2,2))
    

ze_matrix = radar_data_sliced.radar_reflectivity.values
time_radar = pd.to_datetime(radar_data_sliced.time.values)
time_cloud_fraction = np.arange(time_radar[0], time_radar[-1], timedelta(minutes=15))
dim_time_out = len(time_cloud_fraction)
dim_height = np.shape(ze_matrix)[1]
cloud_fraction = np.zeros((dim_time_out, dim_height))
sst_cf = np.zeros((dim_time_out))

for itime in range(dim_time_out-1):
    data = radar_data_sliced.sel(time=slice(time_cloud_fraction[itime], time_cloud_fraction[itime+1]))
    for iheight in range(dim_height):
        cloud_fraction[itime, iheight] = np.count_nonzero(~np.isnan(data.radar_reflectivity.values[:,iheight]))/np.shape(data.radar_reflectivity.values[:,iheight])[0]

    # calculating corresponding mean SST value associated to the 15 min cloud fraction profile. 
    sst_slice = SST_DS.sel(time=slice(time_cloud_fraction[itime], time_cloud_fraction[itime+1]))
    sst_cf[itime] = sst_slice.SST.mean(dim='time', skipna=True)
    
dims = ['time', 'height']
coords = {'time':time_cloud_fraction, 'height':radar_data_sliced.height.values}
cf = xr.DataArray(dims=dims, coords=coords, data=cloud_fraction,
                 attrs={'long_name':'cloud fraction',
                        'units':''})
sst = xr.DataArray(dims=['time'], coords={'time':time_cloud_fraction}, data=sst_cf,
                 attrs={'long_name':'cloud fraction',
                        'units':''})
variables={'cloud_fraction':cf,
          'SST':sst}

cf_DS =xr.Dataset(data_vars = variables,
                       coords = coords)

# calculating mean quantities f
dataset_mean = []
dataset_std = []

# selecting all columns in the bin interval
for ind_bin in range(len(SST_binned_arr_cf)-1):

    # selecting slices of datasets columns with SST values in the selected bin
    DS_sliced = cf_DS.where((cf_DS.SST > SST_binned_arr_cf[ind_bin]) & (cf_DS.SST < SST_binned_arr_cf[ind_bin+1]), drop=True)

    # add variable of the number of elements of the slice
    n_el = len(DS_sliced.SST.values)
    DS_sliced['n_elements'] = n_el

    # calculate mean profile averaging all selected time stamps together
    dataset_mean.append(DS_sliced.mean(dim='time', skipna=True))
    dataset_std.append(DS_sliced.std(dim='time', skipna=True))

# concatenating datasets corresponding to SST bins on a new bin dimension
cloud_fraction_concat = xr.concat([dataset_mean[i] for i in np.arange(len(dataset_mean))], dim='SST_binned')
cloud_fraction_std_concat = xr.concat([dataset_std[i] for i in np.arange(len(dataset_std))], dim='SST_binned')


cloud_fraction_concat.to_netcdf('/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_clau/cloud_fraction_Wband_binned_sst.nc')

